# Comparison between HC$_3$N and NH$_3$

In [ ]:
# original resolution: 4.07672 arcsec x 3.78778 arcsec, pa: -81.4035 deg 
# matching kernel: 4.65325 arcsec x 4.40231 arcsec, pa: 8.59652 deg 
# target resolution: 6 arcsec x 6 arcsec, pa: 0 deg
import numpy as np
from astropy.modeling.models import Gaussian2D
import astropy.units as u
from astropy.nddata import Cutout2D
from astropy.convolution import convolve
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from photutils.psf import create_matching_kernel, TopHatWindow
import matplotlib.pyplot as plt
from reproject import reproject_exact
import aplpy
from B5setup import *
from scipy import stats
from scipy.integrate import simpson
import os
%matplotlib inline

In [ ]:
folderHC3N = 'B5_NOEMA_30m/gaussfit/'
maskfile = folderHC3N + 'B5-NOEMA+30m-H3CN-10-9_cut_K_mask'
paramscubeNH3 = 'B5_previous_data/B5_parameter_maps_snr7'
# for velocity:
filenameHC3Nvlsr = folderHC3N + 'B5-NOEMA+30m-H3CN-10-9_cut_K_1G_fitparams_filtered_Vlsr'
filenameHC3Nvlsr_u = folderHC3N + 'B5-NOEMA+30m-H3CN-10-9_cut_K_1G_fitparams_filtered_Vlsr_unc'
filenameNH3vlsr = 'B5_previous_data/B5_VLA_GBT_model_vc_QA'
diffdatafile = 'B5_Vlsr_HC3N_minus_NH3'
diffdatafile_unc = 'B5_Vlsr_HC3N_minus_NH3_unc'
paramscube_vlsrindex = 4

# For Velocity dispersion :
# filenameHC3Nvlsr = folderHC3N + 'B5-NOEMA+30m-H3CN-10-9_cut_K_1G_fitparams_filtered_SigmaV'
# filenameNH3vlsr = 'B5_previous_data/B5_VLA_GBT_model_dv_QA'
# diffdatafile = 'B5_SigmaV_HC3N_minus_NH3'
# paramscube_sigmaindex = 3

filename_convolved = filenameHC3Nvlsr + '_conv_NH3'
filename_unc_convolved = filenameHC3Nvlsr_u + '_conv_NH3'
filenameNH3_reproject = filenameNH3vlsr + '_reprojectHC3N'
filenameNH3_unc_reproject = filenameNH3vlsr + '_unc_reprojectHC3N'

# alma blue streamer
filenameh2costreamer = 'B5_IRS1_ALMA/gaussfit_H2CO/analysis_central_chans_masked/components_blueshifted_envelope_vlsr'

#to save the minimum points and cuts
min_diff_HC3N_NH3_coords_file = 'min_diff_HC3N_NH3_coords.npy'
hor_cut_file = 'horizontal_cut_HC3N_NH3_diff.npy'
ver_cut_file = 'vertical_cut_HC3N_NH3_diff.npy'

In [ ]:
headerHC3N = fits.getheader(filenameHC3Nvlsr+'.fits')
headerNH3 = fits.getheader(filenameNH3vlsr+'.fits')
wcsHC3N = WCS(headerHC3N)
pixsizeHC3N = np.abs(headerHC3N['CDELT2']) # pixel size in degrees
pixsizeNH3 = np.abs(headerNH3['CDELT2'])

We first need to convolve the HC$_3$N to the resolution of NH$_3$

In [ ]:
if not os.path.exists(filename_convolved + '.fits') or not os.path.exists(filename_unc_convolved + '.fits'):
# we first need to transform all angle units to pixels
    kernelsize = 29
    xcent = int(round(kernelsize/2))

    bmaj_original = headerHC3N['BMAJ']
    bmin_original = headerHC3N['BMIN']
    bpa_original = headerHC3N['BPA'] * u.deg
    sigmamaj_original_pix = (bmaj_original/pixsizeHC3N) / np.sqrt(8*np.log(2))
    sigmamin_original_pix = (bmin_original/pixsizeHC3N) / np.sqrt(8*np.log(2))
    bmaj_target = headerNH3['BMAJ']
    bmin_target = headerNH3['BMIN']
    bpa_target = headerNH3['BPA'] * u.deg
    sigmamaj_target_pix = (bmaj_target/pixsizeHC3N) / np.sqrt(8*np.log(2))
    sigmamin_target_pix = (bmin_target/pixsizeHC3N) / np.sqrt(8*np.log(2))
    
    y, x = np.mgrid[0:kernelsize, 0:kernelsize]
    beamoriginalg = Gaussian2D(100, xcent, xcent, sigmamaj_original_pix, sigmamin_original_pix, theta=bpa_original)
    beamtargetg = Gaussian2D(100, xcent, xcent, sigmamaj_target_pix, sigmamin_target_pix, theta=bpa_target)
    beamoriginal = beamoriginalg(x, y)
    beamtarget = beamtargetg(x, y)
    beamtarget /= np.sum(beamtarget)
    window = TopHatWindow(beta=0.45)
    matchingkernel = create_matching_kernel(beamoriginal, beamtarget, window=window)
    
    mask = fits.getdata(maskfile+'.fits')
    if not os.path.exists(filename_convolved + '.fits'):
        HC3Nvlsr = fits.getdata(filenameHC3Nvlsr+'.fits') 
    
        HC3Nvlsrconvolved = convolve(HC3Nvlsr, matchingkernel)
        HC3Nvlsrconvolved = np.where(mask, HC3Nvlsrconvolved, np.nan)

        newheaderHC3N = headerHC3N.copy()
        newheaderHC3N['BMAJ'] = bmaj_target
        newheaderHC3N['BMIN'] = bmin_target
        newheaderHC3N['BPA'] = bpa_target.value
    
        fits.writeto(filename_convolved+'.fits', HC3Nvlsrconvolved, newheaderHC3N)
        
    if not os.path.exists(filename_unc_convolved + '.fits'):
        HC3Nvlsr_u = fits.getdata(filenameHC3Nvlsr_u+'.fits')
        aprox_sigma_kernel = np.sqrt(sigmamaj_target_pix -sigmamaj_original_pix)
        
        matchingkernel_unc_factor = (2 * np.sqrt(np.pi) * sigmamaj_original_pix * aprox_sigma_kernel / np.sqrt(aprox_sigma_kernel**2 + sigmamaj_original_pix**2))
        matchingkernel_unc = matchingkernel ** 2 * matchingkernel_unc_factor
        HC3Nvlsr_uconvolved = convolve(HC3Nvlsr_u, matchingkernel_unc)
        HC3Nvlsr_uconvolved = np.where(mask, HC3Nvlsr_uconvolved, np.nan)

        newheaderHC3N = headerHC3N.copy()
        newheaderHC3N['BMAJ'] = bmaj_target
        newheaderHC3N['BMIN'] = bmin_target
        newheaderHC3N['BPA'] = bpa_target.value
    
        fits.writeto(filename_unc_convolved+'.fits', HC3Nvlsr_uconvolved, newheaderHC3N)
    HC3Nvlsrconvolved = fits.getdata(filename_convolved + '.fits')
    HC3Nvlsr_uconvolved = fits.getdata(filename_unc_convolved + '.fits')
else:
    HC3Nvlsrconvolved, newheaderHC3N = fits.getdata(filename_convolved + '.fits', header=True)
    HC3Nvlsr_uconvolved = fits.getdata(filename_unc_convolved + '.fits')

Now we need to regrid the NH$_3$ image to the HC$_3$N grid.

In [ ]:
if not os.path.exists(filenameNH3_reproject+'.fits'):
    NH3hdu = fits.open(filenameNH3vlsr + '.fits')
    key_remove = ['PC03_01', 'PC04_01', 'PC01_03', 'PC02_03', 'PC03_02', 'PC04_02','PC03_03', 'PC04_03', 'PC01_04',
                  'PC02_04', 'PC03_04', 'PC04_04', 'CUNIT3', 'CRPIX4','CDELT4','CUNIT4','CTYPE4','CRVAL4','SPECSYS']
    for key_i in key_remove: # there are more dimensions written in the NH3 fits file
        NH3hdu[0].header.remove(key_i)
    newNH3image, footprint = reproject_exact(NH3hdu[0], newheaderHC3N)
    # now we modify the spatial properties of the NH3 Header
    newheaderNH3 = NH3hdu[0].header.copy() # we use this header as it has the resolution
    key_modify = ['CUNIT1', 'CRPIX1','CDELT1','CTYPE1','CRVAL1', 'CUNIT2', 'CRPIX2','CDELT2','CTYPE2','CRVAL2']
    for key_i in key_modify: # there are more dimensions written in the NH3 fits file
        newheaderNH3[key_i] = newheaderHC3N[key_i]

    fits.writeto(filenameNH3_reproject+'.fits', newNH3image, newheaderNH3)
    NH3hdu.close()
else:
    newNH3image,  newheaderNH3 = fits.getdata(filenameNH3_reproject+'.fits', header=True)
    
if not os.path.exists(filenameNH3_unc_reproject+'.fits'):
    params = fits.getdata(paramscubeNH3+'.fits')
    vlsrunc = params[paramscube_vlsrindex+5]
    header = headerNH3.copy()
    key_remove = ['PC03_01', 'PC04_01', 'PC01_03', 'PC02_03', 'PC03_02', 'PC04_02','PC03_03', 'PC04_03', 'PC01_04',
                  'PC02_04', 'PC03_04', 'PC04_04', 'CUNIT3', 'CRPIX4','CDELT4','CUNIT4','CTYPE4','CRVAL4','SPECSYS']
    for key_i in key_remove: # there are more dimensions written in the NH3 fits file
        header.remove(key_i)
    newNH3uncimage, footprint = reproject_exact(fits.PrimaryHDU(vlsrunc, header), newheaderHC3N)
    newheaderNH3unc = header.copy() # we use this header as it has the resolution
    key_modify = ['CUNIT1', 'CRPIX1','CDELT1','CTYPE1','CRVAL1', 'CUNIT2', 'CRPIX2','CDELT2','CTYPE2','CRVAL2']
    for key_i in key_modify: # there are more dimensions written in the NH3 fits file
        newheaderNH3[key_i] = newheaderHC3N[key_i]
    fits.writeto(filenameNH3_unc_reproject+'.fits', newNH3uncimage, newheaderNH3unc)
else:
    newNH3uncimage, newheaderNH3unc = fits.getdata(filenameNH3_unc_reproject+'.fits', header=True)

We look at the difference of HC$_3$N with respect to NH$_3$

In [ ]:
diff = HC3Nvlsrconvolved - newNH3image
newhead = newheaderHC3N.copy()
newhead.remove('BUNIT')
if not os.path.exists(diffdatafile + '.fits'):
    fits.writeto(diffdatafile + '.fits', diff, newhead)


In [ ]:
# we plot quickly the difference
fig = plt.figure(figsize=(4,4))
velmin = -0.2
velmax = 0.2
ax = plot_aplpy_subfig(diffdatafile + '.fits', 
                       fig, (1,1,1), 'linear', velmin, 
                       velmax, 'RdYlBu_r')
ax.add_label(0.6, 0.04, 
             r'$V_{\mathrm{LSR},\mathrm{HC}_3\mathrm{N}(10-9)} - V_{\mathrm{LSR}, \mathrm{NH}_3(1,1)}$', 
             relative=True, family='sans-serif', size=12)
# fig.savefig(folderHC3N + diffdatafile + '.pdf', bbox_inches='tight')

We take a quick look at the uncertainty in the subtraction

In [ ]:
diff_unc = newNH3uncimage + HC3Nvlsr_uconvolved
if not os.path.exists(diffdatafile_unc + '.fits'):
    fits.writeto(diffdatafile_unc + '.fits', diff_unc, newhead)

In [ ]:
# we plot quickly the difference
fig = plt.figure(figsize=(4,4))
velmin = 0
velmax = 0.1
ax = plot_aplpy_subfig(diffdatafile_unc + '.fits', 
                       fig, (1,1,1), 'linear', velmin, 
                       velmax, 'RdYlBu_r')
ax.add_label(0.6, 0.04, 
             r'$V_{\mathrm{LSR},\mathrm{HC}_3\mathrm{N}(10-9)} - V_{\mathrm{LSR}, \mathrm{NH}_3(1,1)}$', 
             relative=True, family='sans-serif', size=12)
ax.colorbar.set_axis_label_text('Uncertainty (km/s)')


Now we do the KDE of the velocity difference distribution, first in 1 dimension and then in 2

In [ ]:
# create the grid for the 1D kernel distribution
if not os.path.exists(diffdatafile+'.npy'):
    data_kde = np.linspace(-0.4, 0.4, 100)
    data = fits.getdata(diffdatafile + '.fits').flatten()
    data = data[~np.isnan(data)]
    p_range = np.array([0.15865 * 100, 50., 0.84135 * 100]) # I believe this is one sigma? ask Jaime
    vlsr_median_unc = np.round(np.percentile(data, p_range), decimals=2)

    kernel = stats.gaussian_kde(data)
    dens = kernel(data_kde)
    dens /= simpson(dens, data_kde) # normalizing of probability

    datasave = np.array([data_kde, dens, vlsr_median_unc], dtype=object)

    np.save(diffdatafile+'.npy', datasave)

In [ ]:
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.hist(data, fc='k', bins='fd', density=True)
# ax.plot(data_kde, dens)
# ax.set_xlim([-0.4, 0.4])
# ax.plot([vlsr_median_unc[1], vlsr_median_unc[1]], [0.5, 2], color='C1', ls='--')
# ax.text(vlsr_median_unc[1]-0.05,1.1,r'{0}$_{{{1:.2f}}}^{{+{2:.2f}}}$'.format(vlsr_median_unc[1], vlsr_median_unc[0]-vlsr_median_unc[1], vlsr_median_unc[2]-vlsr_median_unc[1]), size=10, color='C1')


In [ ]:
# This part only works for velocity
# x is v_LSR of HC3N
if not os.path.exists(diffdatafile+'_2DKDE.npy'):
    xmin = 9.8
    xmax = 10.8
    # y is v_LSR of NH3
    ymin = 9.8
    ymax = 10.8
    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    # we select only those who are not nan
    gd_vlos = np.isfinite(HC3Nvlsrconvolved * newNH3image)
    values = np.vstack([HC3Nvlsrconvolved[gd_vlos], newNH3image[gd_vlos]])
    # we calculate the kernel distribution
    kernel = stats.gaussian_kde(values)
    zz = np.reshape(kernel(positions).T, xx.shape)
    zz /= zz.max()

    datasave = np.array([xx, yy, zz])

    np.save(diffdatafile+'_2DKDE.npy', datasave)


## Cuts along horizontal and vertical samples
We cut a box around the protostar and determine the position of maximum difference to plot the velocities of both molecules in horizontal and vertical cuts

In [ ]:
position_irs1 = SkyCoord(ra_yso*u.deg, dec_yso*u.deg, frame='fk5')
boxsize = 24 # * u.arcsec
boxcut = u.Quantity((boxsize, boxsize), u.arcsec)
# boxsize = 50
wcs_diff = WCS(newhead)
cutout = Cutout2D(diff, position_irs1, boxcut, wcs=wcs_diff)
wcs_diff_small = cutout.wcs
diff_small = cutout.data
boxsize_y, boxsize_x = np.shape(diff_small)
y_min, x_min = np.unravel_index(np.nanargmin(diff_small), diff_small.shape)
x_min_sky, y_min_sky = wcs_diff_small.all_pix2world(x_min, y_min, 0)
x_min_sky, y_min_sky = float(x_min_sky), float(y_min_sky)
if not os.path.exists(min_diff_HC3N_NH3_coords_file):
    np.save(min_diff_HC3N_NH3_coords_file, np.array([x_min_sky, y_min_sky]))

In [ ]:
# we need to cut all images
cutout_unc = Cutout2D(diff_unc, position_irs1, boxcut, wcs=wcs_diff)
diff_unc_small = cutout_unc.data

chansize_hc3n = 0.21 / np.sqrt(2*np.pi)/2.35
chansize_nh3 = 0.04 / np.sqrt(2*np.pi)/2.35 # ask if they are independent
cutout_hc3n = Cutout2D(HC3Nvlsrconvolved, position_irs1, boxcut, wcs=wcs_diff)
hc3nvel_small = cutout_hc3n.data

cutout_hc3n_u = Cutout2D(HC3Nvlsr_uconvolved, position_irs1, boxcut, wcs=wcs_diff)
hc3nvel_small_u = cutout_hc3n_u.data

cutout_nh3 = Cutout2D(newNH3image, position_irs1, boxcut, wcs=wcs_diff)
nh3vel_small = cutout_nh3.data

cutout_nh3_u = Cutout2D(newNH3uncimage, position_irs1, boxcut, wcs=wcs_diff)
nh3vel_small_u = cutout_nh3_u.data

xarray = (np.linspace(0, boxsize_x, boxsize_x) - x_min) * pixsizeHC3N * 3600 # arcsec
yarray = (np.linspace(0, boxsize_y, boxsize_y) - y_min) * pixsizeHC3N * 3600 # arcsec

In [ ]:
# here we get the mask of where the streamer is present
streamer_vel, streamerhead = fits.getdata(filenameh2costreamer+'.fits', header=True)
pixsizeH2CO = streamerhead['CDELT2']
wcs_streamer = WCS(streamerhead)
boxsize_y_stream, boxsize_x_stream = np.shape(streamer_vel)
x_min_stream, y_min_stream = wcs_streamer.all_world2pix(x_min_sky, y_min_sky, 0)
x_min_stream, y_min_stream = int(x_min_stream), int(y_min_stream)
# TODO: Fix to get the correct position with respect to the difference map
x_array_stream = (np.linspace(0, boxsize_x_stream, boxsize_x_stream) - x_min_stream) * pixsizeH2CO * 3600 # arcsec
y_array_stream = (np.linspace(0, boxsize_y_stream, boxsize_y_stream) - y_min_stream) * pixsizeH2CO * 3600 # arcsec

In [ ]:
nh3vls_hor = nh3vel_small[y_min]
nh3vls_hor_u = nh3vel_small_u[y_min]
hc3nvls_hor = hc3nvel_small[y_min]
hc3nvls_hor_u = hc3nvel_small_u[y_min]

nh3vls_ver = nh3vel_small[:, x_min]
nh3vls_ver_u = nh3vel_small_u[:, x_min]
hc3nvls_ver = hc3nvel_small[:, x_min]
hc3nvls_ver_u = hc3nvel_small_u[:, x_min]

diff_hor = diff_small[y_min]
diff_hor_u = diff_unc_small[y_min]
diff_ver = diff_small[:, x_min]
diff_ver_u = diff_unc_small[:, x_min]

streamer_area_hor = ~np.isnan(streamer_vel[y_min_stream])
streamer_area_ver = ~np.isnan(streamer_vel[:,x_min_stream])

In [ ]:
# here e save all the data to plot
cuts_horizontal = np.array([xarray, nh3vls_hor, nh3vls_hor_u, hc3nvls_hor, hc3nvls_hor_u, diff_hor, diff_hor_u, x_array_stream, streamer_area_hor], dtype=object)
cuts_vertical = np.array([yarray, nh3vls_ver, nh3vls_ver_u, hc3nvls_ver, hc3nvls_ver_u, diff_ver, diff_ver_u, y_array_stream, streamer_area_ver], dtype=object)

if not os.path.exists(hor_cut_file): np.save(hor_cut_file, cuts_horizontal)
if not os.path.exists(ver_cut_file): np.save(ver_cut_file, cuts_vertical)

In [ ]:
# the uncertainty for NH3 vlsr is so small that the dominant value comes from HC3N
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(121)
ax.plot(xarray, diff_hor, color='k', marker='.')
ax.fill_between(xarray, diff_hor-diff_hor_u, diff_hor+diff_hor_u, color='k', alpha=0.2)
ax.set_title('Horizontal cut')
ax.set_ylabel(r'$\Delta V_{LSR}$ (km/s)')
ax.fill_between(x_array_stream, 0,1, where=streamer_area_hor, alpha=0.5, transform=ax.get_xaxis_transform())
ax.set_ylim([-0.1, 0.2])
ax2 = fig.add_subplot(122)
ax2.plot(yarray, diff_ver, color='k', marker='.')
ax2.fill_between(yarray, diff_ver-diff_ver_u, diff_ver+diff_ver_u, color='k', alpha=0.2)
ax2.set_title('Vertical cut')
ax2.set_xlabel('Distance from minimum (arcsec)')
ax2.fill_between(y_array_stream, 0,1, where=streamer_area_ver, alpha=0.5, transform=ax2.get_xaxis_transform())
ax2.set_ylim([-0.1, 0.2])

In [ ]:
fig = plt.figure(figsize = (4,4))
ax = fig.add_subplot(111)
ax.errorbar(xarray, hc3nvls_hor, yerr=hc3nvls_hor_u, color='C0', marker='.', linestyle='None')
ax.errorbar(xarray, nh3vls_hor, yerr=nh3vls_hor_u, color='C1', marker='.', linestyle='None')


In [ ]:
fig = plt.figure(figsize = (4,4))
ax = fig.add_subplot(111)
ax.errorbar(yarray, hc3nvls_ver, yerr=hc3nvls_ver_u, color='C0', marker='o', linestyle='None')
ax.errorbar(yarray, nh3vls_ver, yerr=chansize_nh3, color='C1', marker='o', linestyle='None')